In [1]:
import math
import numpy as np
import re
import os.path

In [2]:
# test result function
def frobenius_norm(M1, M2):
    total = 0.0
    for a,b in zip(M1, M2):
        for c, d in zip(a, b):
            total += (c - d)*(c - d)
    return total

In [ ]:
# dimension of each example in the dataset
d = 15
# k is the top pricipal calculated by the eigenvectors, right now just fix it
k = 5
# fix the learning rate
learning_rate = 0.001
# preprocess the data
data_set = []
data_set_done = []
if os.path.exists('new_data'):
    with open('new_data') as data:
        for line in data:
            data_set_done.append(map(float, line.split()))

else:
    with open('adult.data.txt') as data:
        for line in data:
            # '\s' matches whitespace
            tmp = re.sub(r'\s', '', line).split(',')
            data_set.append(tmp)
    print data_set

    feature_vector_size = len(data_set[0])
    print "feature vector is  " ,feature_vector_size
    print "data set size is ", len(data_set)
    tmp = data_set[0]

    # find out which feature is not a digit
    non_digit_index = []
    for i in range (0,feature_vector_size):
        if tmp[i].isdigit():
            continue
        else:
            non_digit_index.append(i)
    print non_digit_index

    # make each string feature to become a number

    # Copy
    modified_data_set= []
    for item in data_set:
        modified_data_set.append(item)


    print "data before processed dimension is ", len(modified_data_set[0])
    #print modified_data_set
    item_list = []
    count = 0
    dimension_to_increase = 0
    # for all the non_digit index, count how many differnt items
    for i in non_digit_index:
        # clear the item list to find different string for same feature
        items = []
        # loop through the whole data set
        for vector in data_set:
            # get the corresponsding item
            tmp = vector[i]
            # it already in the list
            if tmp in items:
                continue
            else:
                # otherwise append to the list
                items.append(tmp)
        dimension_to_increase = dimension_to_increase + len(items)-1
        #print "item is "
        #print items
        #print "\n\n"
        # loop through the whole data set to replace the feature
        for j in range(len(modified_data_set)):
            v = modified_data_set[j]
            #print v
            # get the item in the vector
            #print "accessing ", i+count
            tmp = v[i+count]
            index  = items.index(tmp)
            front = v[0:i+count]
            end = v[i+1+count:len(v)]
            zeros = [0]*len(items)
            zeros[index] = 1
            front.extend(zeros)
            front.extend(end)
            modified_data_set[j] = front
            #print "result is ", modified_data_set[j]
            #v[i] = float(index)
        count = count + len(items)-1
    for v in modified_data_set:
        v = map(float, v)
        data_set_done.append(v)
    print "dimension to increase ", dimension_to_increase
    print "data processed dimension is ", len(data_set_done[0])
    #print data_set_done
    print "size of data set done is ", len(data_set_done)

f = open('new_data', 'w')
for vector in data_set_done:
    for num in vector:
        f.write(str(num)+" ")
    f.write("\n")
f.close()



In [ ]:
# U and V are d x k dimension matrix
v = np.random.rand(len(data_set_done[0]),k)
print "V start with, ", v
u = np.random.rand(len(data_set_done[0]),k)
print "U start with, ", u

print "result of the caulcation is ", np.transpose(data_set_done[0])

M = np.array(data_set_done[0]) * np.transpose(np.array(data_set_done[0]))
for i in range(1, len(data_set_done)):
    M = M + np.array([data_set_done[i]]) * np.transpose(np.array([data_set_done[i]]))
M = M / len(data_set_done)
print "M is "
print M
# SGD function
# run the whole optimization process 10 times
for j in range(0, 10):
    # do u 100 rounds
    for counter in range (0,10):
        print counter
        # update u len(data_set) iterations
        for t in range(0, len(data_set_done)):
            #print (data_set[t] * np.transpose(data_set[t]) - np.dot(u,np.transpose(v)))
            # need to do np.dot() for matrix multiplication
            u = u - learning_rate * np.dot((data_set_done[t] * np.transpose(data_set_done[t]) - np.dot(u,np.transpose(v))), v)
    # do v 100 rounds
    for counter in range (0,10):
        print counter
        # update v len(data_set) iterations
        for t in range(0, len(data_set_done)):
            v = v - learning_rate * np.dot((data_set_done[t] * np.transpose(data_set_done[t]) - np.dot(u, np.transpose(v))), u)
    print frobenius_norm(M, np.dot(u,np.transpose(v)))

V start with,  [[ 0.42982494  0.4715087   0.19394443  0.89804507  0.31381833]
 [ 0.6889543   0.23257854  0.08147234  0.76181677  0.65861444]
 [ 0.4464218   0.82786075  0.55336974  0.75572639  0.02089229]
 [ 0.61747418  0.66769546  0.25240357  0.20119131  0.72155372]
 [ 0.32144359  0.84141992  0.93356202  0.86873367  0.35681042]
 [ 0.8424486   0.14034356  0.74328683  0.49090752  0.17922903]
 [ 0.99628124  0.60070209  0.47948495  0.09243757  0.88049848]
 [ 0.33091827  0.9177726   0.31570561  0.51666818  0.14420005]
 [ 0.22145604  0.5514179   0.48925484  0.2912927   0.57216537]
 [ 0.0970801   0.60350563  0.83486383  0.55015595  0.03652324]
 [ 0.30587843  0.62965335  0.6259132   0.38303082  0.34852628]
 [ 0.02805373  0.7434881   0.3936925   0.50489397  0.01278789]
 [ 0.36744354  0.37026558  0.15760033  0.36387511  0.19293795]
 [ 0.89430243  0.67049239  0.75753054  0.3144884   0.28015678]
 [ 0.4493962   0.28661544  0.3641435   0.6639252   0.87597146]
 [ 0.93995212  0.68807555  0.23010777  0